In [2]:
import spotipy
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud
import random

# Data Gathering

In [33]:
# Spotify API tokens for access
from spotipy.oauth2 import SpotifyClientCredentials

client_id= 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'
client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [34]:
# function to know how long it'll take to scrape the data

def format_time(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

In [35]:
# create empty lists to store data
artist_name = []
track_name = []
track_id = []
album_name = []
album_id = []
release_date = []
duration_ms = []
popularity = []
explicit = []
danceability = []
energy = []
key = []
loudness = []
genres = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
time_signature = []
featured_artists = []

In [ ]:
# start time
start_time = timer()

# For Ed sheeran as a main artist

# loop through results, using offset to get all tracks
for i in range(0, 1000, 50):
    track_results = sp.search(q='artist:ed sheeran', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        
        # prevent live performance tracks from entering the dataset
        if "live" not in t['name'].lower() and "live" not in t['album']['name'].lower():
            # get track details
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            album_name.append(t['album']['name'])
            album_id.append(t['album']['id'])
            release_date.append(t['album']['release_date'])
            popularity.append(t['popularity'])
            explicit.append(t['explicit'])

            # get audio features for track
            audio_features = sp.audio_features(t['id'])[0]
            danceability.append(audio_features['danceability'])
            duration_ms.append(audio_features['duration_ms'])
            energy.append(audio_features['energy'])
            key.append(audio_features['key'])
            loudness.append(audio_features['loudness'])
            mode.append(audio_features['mode'])
            speechiness.append(audio_features['speechiness'])
            acousticness.append(audio_features['acousticness'])
            instrumentalness.append(audio_features['instrumentalness'])
            liveness.append(audio_features['liveness'])
            valence.append(audio_features['valence'])
            tempo.append(audio_features['tempo'])
            time_signature.append(audio_features['time_signature'])

            
            # get featured artists
            if len(t['artists']) > 1:
                feat_artists = []
                for j in range(1, len(t['artists'])):
                    feat_artists.append(t['artists'][j]['name'])
                featured_artists.append(feat_artists)
            else:
                featured_artists.append([])
              
                
                
# create dataframe from lists
df = pd.DataFrame({
    'artist_name': artist_name,
    'track_name': track_name,
    'track_id': track_id,
    'album_name': album_name,
    'album_id': album_id,
    'release_date': release_date,
    'duration_ms': duration_ms,
    'popularity': popularity,
    'explicit': explicit,
    'danceability': danceability,
    'energy': energy,
    'key': key,
    'loudness': loudness,
    'mode': mode,
    'speechiness': speechiness,
    'acousticness': acousticness,
    'instrumentalness': instrumentalness,
    'liveness': liveness,
    'valence': valence,
    'tempo': tempo,
    'time_signature': time_signature,
    'featured_artists': featured_artists
})



# end time and print
end_time = timer()
elapsed_time = int(end_time - start_time)
print(f"Elapsed time: {format_time(elapsed_time)}")

In [ ]:
# store dataframe as csv file
df.to_csv('Ed Sheeran Spotify Tracks.csv', index=False)

---
# Data Assessment

In [199]:
# loading the metadata of the tracks that exist as at the time this notebook is being written

df = pd.read_csv('Ed Sheeran Spotify Tracks.csv')

In [200]:
df.info()

shape = df.shape
print(f"\n\n The data set has {shape[0]} rows and {shape[1]} columns.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       978 non-null    object 
 1   track_name        978 non-null    object 
 2   track_id          978 non-null    object 
 3   album_name        978 non-null    object 
 4   album_id          978 non-null    object 
 5   release_date      978 non-null    object 
 6   duration_ms       978 non-null    int64  
 7   popularity        978 non-null    int64  
 8   explicit          978 non-null    bool   
 9   danceability      978 non-null    float64
 10  energy            978 non-null    float64
 11  key               978 non-null    int64  
 12  loudness          978 non-null    float64
 13  mode              978 non-null    int64  
 14  speechiness       978 non-null    float64
 15  acousticness      978 non-null    float64
 16  instrumentalness  978 non-null    float64
 1

In [201]:
# View first 15 rows with all columns
pd.options.display.max_columns = None 
df.head(15)

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Ed Sheeran,Shivers,50nfwKoDiSYg8zOCREWAm5,=,32iAEBstCjauDhyKpGjTuq,2021-10-29,207853,83,False,0.788,0.859,2,-2.724,1,0.0856,0.28100,0.000000,0.0424,0.822,141.020,4,[]
1,Ed Sheeran,Perfect,0tgVpDi06FyKpA1z0VMD4v,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,263400,85,False,0.599,0.448,8,-6.312,1,0.0232,0.16300,0.000000,0.1060,0.168,95.050,3,[]
2,Ed Sheeran,Bad Habits,3rmo8F54jFF8OgYsqTxm5d,=,32iAEBstCjauDhyKpGjTuq,2021-10-29,230747,83,False,0.807,0.893,11,-3.745,0,0.0347,0.04510,0.000028,0.3660,0.537,126.011,4,[]
3,Ed Sheeran,Tenerife Sea,1WTY0VL681yvPxvUuJ5GXY,Intimate Pop,65qTx2hiYqEvBu192EoV8H,2023-02-24,241347,1,False,0.526,0.345,8,-10.497,1,0.0374,0.70000,0.000011,0.1050,0.357,121.877,4,[]
4,Ed Sheeran,Perfect,4oAhMSYRMl1SAv02GbG46I,The Words - Soft 10s,2heLXJ9F1z8P8ypjyale5D,2023-02-21,263400,1,False,0.598,0.448,8,-6.312,1,0.0232,0.16200,0.000000,0.1060,0.168,95.009,3,[]
5,Ed Sheeran,Overpass Graffiti,1QkYsNB4TA2NJwq4kykl8L,Hit It Up,3ReRqGIICTRYl9cGy5UNzL,2023-02-23,236907,0,False,0.520,0.849,9,-3.794,0,0.1860,0.00253,0.000000,0.1190,0.774,175.908,4,[]
6,Ed Sheeran,Noche de novela,7tgdYuyG7vyuQWwUJRAFuC,2022 Rewind,3wKJ0U1gF1yqbk9tLxz6j5,2023-02-22,191896,0,False,0.522,0.585,7,-7.205,1,0.1290,0.00208,0.000000,0.0909,0.411,97.990,4,['Paulo Londra']
7,Ed Sheeran,Shivers,5UuXHpsnE4KKPaVcD1P5JQ,All That Really Matters: Hits Now,2O6uwpC2iC6ZVaaCZpJ2Wy,2023-02-23,207853,0,False,0.788,0.859,2,-2.724,1,0.0856,0.28100,0.000000,0.0424,0.822,141.020,4,[]
8,Ed Sheeran,Noche de novela,5SxjEDljqUFpkw2UNMz7tB,Mega Pop Mix,2pxZyFVNAh8N7AlVfDJYFO,2023-02-22,191896,0,False,0.522,0.585,7,-7.205,1,0.1290,0.00208,0.000000,0.0909,0.411,97.990,4,['Paulo Londra']
9,Ed Sheeran,Photograph,3ATeaAvchVwa80bSg2fiF6,Love Like That - Feeling Blue,3mqe2Z1a4UjDYdxYZtXeCZ,2023-02-18,258987,0,False,0.718,0.379,4,-10.480,1,0.0359,0.60700,0.000472,0.0986,0.220,108.033,4,[]


In [202]:
# A quick glance shows that "Noche de novela" appears more than once. INvestigating further,

df[df['track_name'] == 'Noche de novela']

# gives the same result as ** df[df['track_name'].str.contains('Noche de novela')] **

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
6,Ed Sheeran,Noche de novela,7tgdYuyG7vyuQWwUJRAFuC,2022 Rewind,3wKJ0U1gF1yqbk9tLxz6j5,2023-02-22,191896,0,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']
8,Ed Sheeran,Noche de novela,5SxjEDljqUFpkw2UNMz7tB,Mega Pop Mix,2pxZyFVNAh8N7AlVfDJYFO,2023-02-22,191896,0,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']
26,Ed Sheeran,Noche de novela,02u3v7PgAQG4M8a2wqoSSy,Tropical Hits,2tqbyN2aAatZLagWNTwHEn,2023-02-21,191896,0,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']
51,Ed Sheeran,Noche de novela,6Zc1FpeuOj41jqWS8nF1eK,Some Latin Hits,1fBCs2MTAlN9DMmHBfAdcQ,2023-02-16,191896,1,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']
124,Ed Sheeran,Noche de novela,3mm8QyStd0mbBSzX2WQ0aa,Reggaetón Éxitos,07TLih8JBIJYv5BBov38hy,2023-02-20,191896,1,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']
128,Ed Sheeran,Noche de novela,2ZazX4EQfZrkWoca0EZwEW,Hot Latin Hits,0NvLdTDrIySgNFqUvtgEj4,2023-02-16,191896,1,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']
133,Ed Sheeran,Noche de novela,2F5Bk49TmkDIOHEKpgEcso,Latin Hits 20's,29ZDCi9G0z6aLJvTI5cNfK,2023-02-21,191896,0,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']
824,Ed Sheeran,Noche de novela,6vIEv5l5BbGxXTsS8cO1Kw,Hot Reggaeton 20s,6CiYJOb4Vo1irNUHxGvuXe,2023-02-21,191896,0,False,0.522,0.585,7,-7.205,1,0.129,0.00208,0.0,0.0909,0.411,97.99,4,['Paulo Londra']


In [203]:
df[df['track_name'] == 'Perfect'].sort_values(by='popularity', ascending=False).head()

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
1,Ed Sheeran,Perfect,0tgVpDi06FyKpA1z0VMD4v,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,263400,85,False,0.599,0.448,8,-6.312,1,0.0232,0.163,0.000,0.1060,0.168,95.050,3,[]
268,Ed Sheeran,Perfect,2R0FW1n6cOkQDJhAkQENsf,Perfect,34MK1MIhRB9evwfomSyMig,2017-07-21,310799,46,False,0.538,0.215,8,-13.881,1,0.0608,0.979,0.901,0.0876,0.112,179.199,3,['The Piano Guys']
541,Ed Sheeran,Perfect,0vLpDTq6TGxKI5xPAwoW2F,Why Can't We Be Friends?,5JZUPjOHK53a9n68f5Djqn,2021-03-05,263400,29,False,0.598,0.448,8,-6.312,1,0.0232,0.162,0.000,0.1060,0.168,95.009,3,[]
345,Ed Sheeran jr,Perfect,0KkTaYH2Szr70G5YZLgA6z,Perfect,5EVOYJ2SEgusyCiOw1XVAr,2022-09-06,61984,28,False,0.734,0.167,8,-19.098,0,0.0461,0.854,0.718,0.1650,0.117,110.114,4,[]
534,Ed Sheeran,Perfect,6hE7M9H7AWVvXHlaMoMb3e,Talking to the Moon - Soft Pop,0hl8eJX2PykIhIPBI0AlRh,2021-04-23,263400,19,False,0.598,0.448,8,-6.312,1,0.0232,0.162,0.000,0.1060,0.168,95.009,3,[]


In [204]:
track_perfect = df[df['track_name'] == 'Perfect']
track_perfect[df.track_id.duplicated(keep = False)]

C:\Users\Zion\AppData\Local\Temp\ipykernel_19240\897602886.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_perfect[df.track_id.duplicated(keep = False)]


,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
676,Ed Sheeran,Perfect,09oCjFhI30LRKXtGyJjkPl,Chill Hits 2022,7EwibYmDDr4ZSWMVUhlIdX,2022-07-01,263400,5,False,0.598,0.448,8,-6.312,1,0.0232,0.162,0.0,0.106,0.168,95.009,3,[]
680,Ed Sheeran,Perfect,09oCjFhI30LRKXtGyJjkPl,Chill Hits 2022,7EwibYmDDr4ZSWMVUhlIdX,2022-07-01,263400,5,False,0.598,0.448,8,-6.312,1,0.0232,0.162,0.0,0.106,0.168,95.009,3,[]


In [205]:
df.album_name.sort_values(ascending=True).unique()

array(['+', '1, 2, 3 - Viral Hits', "10's Soft Pop", '100 Easy Hits',
       '100 Greatest Christmas Songs Ever',
       '100 Greatest Dinner Party Songs', '100 Greatest Driving Songs',
       '100 Greatest Party Songs', '100 Love Songs Classic Hits',
       '100 Most Uplifting Songs Ever',
       '1000 Nights (feat. Meek Mill & A Boogie wit da Hoodie)',
       '10:35 - time for hits', '10s Hits', '10s Pop',
       "20's Biggest Bops", "20's Love Songs", "20's Party",
       "20's Pop Songs", "20's Rock", '2022 Rewind', '20er Hits',
       '20s Finest Pop', '21st Century Pop', '2step (Star.One Remix)',
       '2step (The Remixes)', '2step (feat. 1.Cuz)',
       '2step (feat. Antytila)', '2step (feat. Armaan Malik)',
       '2step (feat. Budjerah)', '2step (feat. Chefin)',
       '2step (feat. Denise Chaila)', '2step (feat. Ellinoora)',
       '2step (feat. Leto)', '2step (feat. Lil Baby)',
       '2step (feat. Potter Payper)', '2step (feat. Quevedo)',
       '2step (feat. Ultimo)', '2s

There are 73 tracks by Ed SHeeran titled Perfect, with 2 having the same track ID. This is a unique case of duplicate values, and is similar to the track "Noche de novela". Definetely, there will be other tracks like this.

In [206]:
df[df['album_name'].str.contains('Deluxe')]

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
1,Ed Sheeran,Perfect,0tgVpDi06FyKpA1z0VMD4v,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,263400,85,False,0.599,0.448,8,-6.312,1,0.0232,0.1630,0.000000,0.1060,0.168,95.050,3,[]
126,Ed Sheeran,2step,1n4t8sZsyyHeIsJAUY7Mka,R'n'B Deluxe,41DSTx5MSUkNNvjz55j0lI,2023-02-22,163450,0,False,0.693,0.643,4,-5.867,0,0.0828,0.1380,0.000000,0.1080,0.477,95.055,4,['Lil Baby']
142,Ed Sheeran,Shape of You,7qiZfU4dY1lWllzX7mPBI3,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,233713,83,False,0.825,0.652,1,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,4,[]
143,Ed Sheeran,Thinking out Loud,34gCuhDGsG4bRPIf9bb02f,x (Deluxe Edition),1xn54DMo2qIqBuMqHtUsFd,2014-06-21,281560,80,False,0.781,0.445,2,-6.061,1,0.0295,0.4740,0.000000,0.1840,0.591,78.998,4,[]
144,Ed Sheeran,Photograph,1HNkqx9Ahdgi1Ixy2xkKkL,x (Deluxe Edition),1xn54DMo2qIqBuMqHtUsFd,2014-06-21,258987,82,False,0.614,0.379,4,-10.480,1,0.0476,0.6070,0.000464,0.0986,0.201,107.989,4,[]
145,Ed Sheeran,Castle on the Hill,6PCUP3dWmTjcTtXY02oFdT,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,261154,76,False,0.461,0.834,2,-4.868,1,0.0989,0.0232,0.000011,0.1400,0.471,135.007,4,[]
146,Ed Sheeran,Happier,2RttW7RAu5nOAfq6YFvApB,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,207520,76,False,0.522,0.385,0,-7.355,1,0.0288,0.5360,0.000000,0.1350,0.236,89.792,4,[]
147,Ed Sheeran,Galway Girl,0afhq8XCExXpqazXczTSve,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,170827,76,False,0.624,0.876,9,-3.374,1,0.1000,0.0735,0.000000,0.3270,0.781,99.943,4,[]
156,Ed Sheeran,Supermarket Flowers,4VuS959DSpr82t3qBqCrWG,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,221107,72,False,0.589,0.242,6,-10.517,1,0.0442,0.9140,0.000000,0.0887,0.257,89.749,4,[]
160,Ed Sheeran,Dive,51ChrwmUPDJvedPQnIU8Ls,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,238440,70,False,0.761,0.386,4,-6.158,1,0.0399,0.3550,0.000000,0.0953,0.526,134.943,3,[]


In [207]:
df.featured_artists.sample(n=20, random_state=123)

581                           []
881                           []
368               ['Elton John']
714                           []
85                            []
381                           []
221                           []
704                           []
156                           []
145                           []
908                           []
415                 ['Lil Baby']
914                           []
456                  ['P Money']
375                           []
343    ['FEDUK', 'SLAVA MARLOW']
584                           []
775                 ['Lil Baby']
308                           []
663                 ['Lil Baby']
Name: featured_artists, dtype: object

In [208]:
df.artist_name.unique()

array(['Ed Sheeran', 'Ed Sheeran & Justin Bieber', 'Ed Sheeran jr',
       'Edward Christopher Sheeran', 'Ed Sheeran Taylor Swift',
       'No. 1 Ed Sheeran Fan', 'Ed Sheeran Band', 'Eddi Silver'],
      dtype=object)

In [209]:
df[df['artist_name'] != 'Ed Sheeran'].head(50)

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
152,Ed Sheeran & Justin Bieber,I Don't Care (with Justin Bieber),3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),5Nux7ozBJ5KJ02QYWwrneR,2019-05-10,219947,75,False,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,4,['Justin Bieber']
174,Ed Sheeran & Justin Bieber,I Don't Care (with Justin Bieber),0hVXuCcriWRGvwMV1r5Yn9,No.6 Collaborations Project,3oIFxDIo2fwuk4lwCmFZCx,2019-07-12,219947,68,False,0.798,0.676,6,-5.041,1,0.0442,0.09020,0.000000,0.0894,0.843,101.956,4,['Justin Bieber']
256,Ed Sheeran jr,Chiraq,0WlIKx95bYbp5FSyEGDgf8,Chiraq,2tVSowJb30U0wIHkPsjFQo,2022-12-27,61138,39,False,0.569,0.945,3,-2.324,0,0.0317,0.13900,0.329000,0.3580,0.931,140.017,4,[]
258,Ed Sheeran jr,Apru,1GZEMY8Rxp265vPoE0rCaB,Apru,37dfnupAknLCPdxELz4MVZ,2022-12-27,63019,39,False,0.593,0.829,11,-2.198,1,0.0520,0.00216,0.885000,0.6800,0.552,140.006,4,[]
313,Ed Sheeran jr,Happier,2mXMy429ZGpZLqDitZNXNQ,Happier,6ljseTw39aY2F1HegpiDP1,2022-09-06,83607,31,False,0.782,0.399,9,-9.537,1,0.3000,0.86700,0.915000,0.1060,0.543,192.029,4,[]
345,Ed Sheeran jr,Perfect,0KkTaYH2Szr70G5YZLgA6z,Perfect,5EVOYJ2SEgusyCiOw1XVAr,2022-09-06,61984,28,False,0.734,0.167,8,-19.098,0,0.0461,0.85400,0.718000,0.1650,0.117,110.114,4,[]
373,Ed Sheeran & Justin Bieber,I Don't Care (with Justin Bieber),1Tu5fpEjGl78R4mA6pQ7Tk,No.6 Collaborations Project,5oUZ9TEZR3wOdvqzowuNwl,2019-07-12,219947,37,False,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,4,['Justin Bieber']
396,Ed Sheeran jr,Shivers,08CIuzA2hK60XSIl8Itg24,Shivers,63PQcM7djBE00HnDuxlPr1,2022-09-06,70290,23,False,0.709,0.315,5,-13.703,0,0.0439,0.56800,0.913000,0.1040,0.615,149.981,4,[]
529,Edward Christopher Sheeran,Perfect (Arr. for 4 Marimbas),61jjBiwRR0h5xQpoikHGf1,Carmen,5EZXMPhBSrGqW0cpQZ0ibd,2019-09-13,288280,14,False,0.491,0.129,8,-15.460,1,0.0606,0.98400,0.881000,0.0683,0.101,104.021,4,['The Wave Quartet']
602,Ed Sheeran Taylor Swift,The Joker & The Queen,4UvVC1xk0SsRjqiY23Hzqf,"Cover Selections, Vol. 2",50D2189Z9geTW7ERTFB8MZ,2022-10-30,217551,8,False,0.117,0.215,0,-11.835,1,0.0392,0.94500,0.367000,0.0954,0.176,154.684,5,['Amethyst String Quartet']


In [210]:
# check tracks with duplicate track ID
df[df.track_id.duplicated(keep = False)]

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
676,Ed Sheeran,Perfect,09oCjFhI30LRKXtGyJjkPl,Chill Hits 2022,7EwibYmDDr4ZSWMVUhlIdX,2022-07-01,263400,5,False,0.598,0.448,8,-6.312,1,0.0232,0.162,0.0,0.106,0.168,95.009,3,[]
677,Ed Sheeran,2step (feat. Potter Payper),6P0lfcQhycP2MEx8QUBFlH,2step (The Remixes),1nqJFaKVTrCkCDaC2I1zM7,2022-05-26,153447,10,False,0.726,0.671,4,-5.402,0,0.1610,0.140,0.0,0.139,0.552,95.053,4,['Potter Payper']
680,Ed Sheeran,Perfect,09oCjFhI30LRKXtGyJjkPl,Chill Hits 2022,7EwibYmDDr4ZSWMVUhlIdX,2022-07-01,263400,5,False,0.598,0.448,8,-6.312,1,0.0232,0.162,0.0,0.106,0.168,95.009,3,[]
681,Ed Sheeran,2step (feat. Potter Payper),6P0lfcQhycP2MEx8QUBFlH,2step (The Remixes),1nqJFaKVTrCkCDaC2I1zM7,2022-05-26,153447,10,False,0.726,0.671,4,-5.402,0,0.1610,0.140,0.0,0.139,0.552,95.053,4,['Potter Payper']


In [211]:
df.album_id.duplicated().sum()

302

In [212]:
df[df.album_id.duplicated(keep = False)].album_id.unique().size

97

There are 97 album IDs appearing 302 times. For Album ID's, It is expected to have duplicate values as many tracks from one album would all have the same Album ID. AFter further cleaning, it is expected that redundant, duplicate IDs would've been removed

In [213]:
df.album_name.unique().size, df.album_id.unique().size

(661, 676)

In [214]:
df[['track_name', 'track_id', 'album_name', 'album_id']].isna().any()

track_name    False
track_id      False
album_name    False
album_id      False
dtype: bool

---

### Quality Issues

1. There are some tracks that are duplicate. they appear on different albums (e.g deluxes), as singles, and some as covers. e.g **Noche de novela** appearing under **2022 Rewind**, **Mega Pop Mix** and 6 other "albums" across 4 days.
2. Duration_ms should have it's values in minutes and seconds, as oppossed to milliseconds for mor readability
3. Not all enteries under artist_name have "Ed Sheeran" as their value.
4. 



### Tidiness Issues
1. Apart from the release date column, there should be column for month and year.
2.

---

# Cleaning Data

## Issue 1

Duplicate tracks

### Code

In [215]:
# get all duplicates by track name
duplicates = df[df.duplicated(subset=['track_name'], keep=False)]

# create a list of album names to keep
keep_albums = ["x (Deluxe Edition)", "÷ (Deluxe)", "=", "+", "No.5 Collaborations Project", "No.6 Collaborations Project"]

# drop duplicates with album names not in the keep_albums list
duplicates_to_drop = duplicates[~duplicates['album_name'].isin(keep_albums)]

# drop duplicates to keep the highest rated track
duplicates_to_keep = duplicates.drop(duplicates_to_drop.index)\
                    .sort_values('popularity', ascending=False).drop_duplicates(subset=['track_name'])

# combine the two dataframes, dropping the duplicates that should be dropped
df = pd.concat([df.drop(duplicates_to_drop.index), duplicates_to_keep])

df.drop_duplicates(subset=['track_name'], keep='first', inplace=True)

### Test

In [221]:
df['track_name'].duplicated().sum()

0

## Issue 2

Correct artist name to have only Ed Sheeran enteries.

### Code

In [217]:
artist_list = df.artist_name.unique()
artist_list

array(['Ed Sheeran', 'Ed Sheeran & Justin Bieber', 'Ed Sheeran jr',
       'Ed Sheeran Taylor Swift', 'No. 1 Ed Sheeran Fan',
       'Edward Christopher Sheeran', 'Ed Sheeran Band'], dtype=object)

In [ ]:
# add 'Taylor Swift' to the featured_artist column for 'Ed Sheeran Taylor Swift' row
mask = df['artist_name'] == 'Ed Sheeran Taylor Swift'
index = df.index[mask][0]  # get the index of the first row where mask is True
df.at[index, 'featured_artists'] = ['\'Taylor Swift\'', '\'Amethyst String Quartet\'']

# replace 'Ed Sheeran & Justin Bieber' and 'Ed Sheeran Taylor Swift' with 'Ed Sheeran'
df['artist_name'] = df['artist_name'].replace(['Ed Sheeran & Justin Bieber', 'Ed Sheeran Taylor Swift'], 'Ed Sheeran')

# drop rows with unwanted artist_names
df = df[~df['artist_name'].isin(['No. 1 Ed Sheeran Fan', 'Edward Christopher Sheeran', 'Ed Sheeran Band', 'Ed Sheeran jr'])]

### Test

In [220]:
df.artist_name.unique()

array(['Ed Sheeran'], dtype=object)

## Issue 3

'duration_ms' should have it's values in minutes and seconds

### Code

In [222]:
# convert the duration column to a timedelta format
df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')

# extract the minutes and seconds components of the duration as strings
df['duration_ms'] = df['duration_ms'].dt.components['minutes'].astype(str).str.zfill(2) + ':' + df['duration_ms'].dt.components['seconds'].astype(str).str.zfill(2)

### Test

In [226]:
df.duration_ms.head()

0      03:27
1      04:23
2      03:50
142    03:53
143    04:41
Name: duration_ms, dtype: object

## Issue 4

Release date should have day month and year columns

In [227]:
# convert release_date column to datetime format
df['release_date'] = pd.to_datetime(df['release_date'])

# extract month and year from release_date column
df['release_day'] = df['release_date'].dt.strftime('%A')
df['release_month'] = df['release_date'].dt.strftime('%B')
df['release_year'] = df['release_date'].dt.year

### Test

In [229]:
df.head()

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists,release_day,release_month,release_year
0,Ed Sheeran,Shivers,50nfwKoDiSYg8zOCREWAm5,=,32iAEBstCjauDhyKpGjTuq,2021-10-29,03:27,83,False,0.788,0.859,2,-2.724,1,0.0856,0.2810,0.000000,0.0424,0.822,141.020,4,[],Friday,October,2021
1,Ed Sheeran,Perfect,0tgVpDi06FyKpA1z0VMD4v,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,04:23,85,False,0.599,0.448,8,-6.312,1,0.0232,0.1630,0.000000,0.1060,0.168,95.050,3,[],Friday,March,2017
2,Ed Sheeran,Bad Habits,3rmo8F54jFF8OgYsqTxm5d,=,32iAEBstCjauDhyKpGjTuq,2021-10-29,03:50,83,False,0.807,0.893,11,-3.745,0,0.0347,0.0451,0.000028,0.3660,0.537,126.011,4,[],Friday,October,2021
142,Ed Sheeran,Shape of You,7qiZfU4dY1lWllzX7mPBI3,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,03:53,83,False,0.825,0.652,1,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,4,[],Friday,March,2017
143,Ed Sheeran,Thinking out Loud,34gCuhDGsG4bRPIf9bb02f,x (Deluxe Edition),1xn54DMo2qIqBuMqHtUsFd,2014-06-21,04:41,80,False,0.781,0.445,2,-6.061,1,0.0295,0.4740,0.000000,0.1840,0.591,78.998,4,[],Saturday,June,2014


---
# Storing Data

In [230]:
# Store dataset
df.to_csv('Ed Sheeran Spotify Tracks Cleaned.csv', index=False)

---
# Analyzing and Visualizing Data

In [231]:
# Load dataframe for analysis
df = pd.read_csv('Ed Sheeran Spotify Tracks Cleaned.csv')